# スタッキング003

In [1]:
import torch
# GPUの名称、CUDA Compute Capabilityを取得
print(torch.cuda.get_device_name())

NVIDIA GeForce RTX 2080 Ti


In [2]:
import cupy as cp
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import scipy.stats as st
import pickle
import optuna
import re
import gc

from lightgbm import LGBMClassifier

from sklearn import metrics
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.impute import SimpleImputer
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.metrics import roc_auc_score
from sklearn.datasets import make_moons
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import mutual_info_classif
from sklearn.experimental import enable_iterative_imputer

from keras.layers import Dense, Dropout
from keras.models import Sequential

from six import StringIO

from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import math
from scipy import stats
import numpy as np
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow import feature_column
from tensorflow.keras import layers

from tqdm import tqdm

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
from sklearn.metrics import log_loss
from xgboost.sklearn import XGBClassifier

from Modules import Modules
from Baseline import Baseline
from FeatureEngineering import FeatureEngineering
from FeatureEngineering2 import FeatureEngineering2
from FeatureEngineering3 import FeatureEngineering3

from models import Model1Xgb, Model1Lgb, Model1NN, Model2Linear

%matplotlib inline
gc.enable()

print("OK\n")

OK



## 共通関数定義

In [3]:
### LightGBM
params_lgb = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'num_leaves': 58
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_weight': 6
    ,'min_child_samples': 165
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}
### Xgboost
params_xgb = {
        'booster': 'gbtree'
        ,'objective': 'binary:logistic'
        ,'silent': 1
        ,'random_state': 17
        ,'eval_metric': 'auc'
        ,'eta': 0.1
        ,'max_depth': 8
        ,'subsample': 0.9
        ,'colsample_state': 71
        ,'colsample_bytree': 1.0
        ,'colsample_bylevel': 0.3
        ,'gamma': 0
        ,'lambda': 1
        ,'alpha': 0
        ,'min_child_weight': 1
        ,'gpu_id': 0
        ,'tree_method': 'gpu_hist'
}

In [4]:
# 学習データに対する「目的変数を知らない」予測値と、テストデータに対する予測値を返す関数
def predict_cv(model, train_x, train_y, test_x):
    preds = []
    preds_test = []
    va_idxes = []

    kf = KFold(n_splits=4, shuffle=True, random_state=71)

    # クロスバリデーションで学習・予測を行い、予測値とインデックスを保存する
    for i, (tr_idx, va_idx) in enumerate(kf.split(train_x)):
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
        model.fit(tr_x, tr_y, va_x, va_y)
        pred = model.predict(va_x)
        print(pred)
        preds.append(pred)
        pred_test = model.predict(test_x)
        preds_test.append(pred_test)
        va_idxes.append(va_idx)

    # バリデーションデータに対する予測値を連結し、その後元の順序に並べ直す
    va_idxes = np.concatenate(va_idxes)
    preds = np.concatenate(preds, axis=0)
    order = np.argsort(va_idxes)
    pred_train = preds[order]

    # テストデータに対する予測値の平均をとる
    preds_test = np.mean(preds_test, axis=0)

    return pred_train, preds_test

In [5]:
params_base = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread': -1
    ,'n_estimators': 5000
    ,'learning_rate': 0.01
    ,'max_depth': 11
    ,'colsample_bytree': 0.613
    ,'subsample': 0.708
    ,'max_bin': 407
    ,'reg_alpha': 3.564
    ,'reg_lambda': 4.930
    ,'min_child_samples': 165
    ,'silent': -1
    ,'verbose': -1
    ,'importance_type': 'gain'
}



def objective(trial):
    # 探索するパラメータ
    param_tuning = {
            "num_leaves": trial.suggest_int("num_leaves", 8, 256)
            ,"min_child_samples": trial.suggest_int("min_child_samples", 5, 200)
            ,"min_sum_hessian_in_leaf": trial.suggest_float("min_sum_hessian_in_leaf", 1e-5, 1e-2, log=True)
            ,"feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1.0)
            ,"bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1.0)
            ,"lambda_l1": trial.suggest_float("lambda_l1", 1e-2, 1e+2, log=True)
            ,"lambda_l2": trial.suggest_float("lambda_l2", 1e-2, 1e+2, log=True)
    }
    param_tuning.update(params_base)

    # モデル学習・評価:
    list_metrics = []
    cv = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(x_train, y_train))
    list_fold = [0]
    for nfold in list_fold:
        idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
        x_tr, y_tr = x_train.loc[idx_tr, :], y_train[idx_tr]
        x_va, y_va = x_train.loc[idx_va, :], y_train[idx_va]
        model = lgb.LGBMClassifier(**params)
        model.fit(x_tr
                    ,y_tr
                    ,eval_set=[(x_tr, y_tr), (x_va, y_va)]
                    ,early_stopping_rounds=100
                    ,verbose=0
        )
        y_va_pred = model.predict_proba(x_va)[:,1]
        metric_va = roc_auc_score(y_va, y_va_pred)
        list_metrics.append(metric_va)

    # 評価指数の算出
    metrics = np.mean(list_metrics)

    return metrics

## インスタンス生成

In [6]:
baseline = Baseline('SK_ID_CURR')
modules = Modules('SK_ID_CURR')
FeatureEngineering = FeatureEngineering('TARGET')
FeatureEngineering2 = FeatureEngineering2('TARGET')
FeatureEngineering3 = FeatureEngineering3('TARGET')

## 特徴量生成1

In [7]:
apps, prev, bureau, bureau_bal, pos_bal, install, card_bal = FeatureEngineering3.get_dataset()
apps.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1.0,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0.0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0.0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0.0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0.0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
df = FeatureEngineering3.get_apps_all_with_all_agg(apps, prev, bureau, bureau_bal, pos_bal, install, card_bal)
df.head()

prev_agg shape: (338857, 41) bureau_agg shape: (305811, 103)
pos_bal_agg shape: (337252, 45) install_agg shape: (339587, 59) card_bal_agg shape: (103558, 70)
apps_all before merge shape: (356255, 152)
apps_all after merge with all shape: (356255, 466)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CARD_M3BALANCE_LIMIT_RATIO_MIN,CARD_M3BALANCE_LIMIT_RATIO_MAX,CARD_M3DRAWING_LIMIT_RATIO_MIN,CARD_M3DRAWING_LIMIT_RATIO_MAX,CARD_M3CARD_IS_DPD_MEAN,CARD_M3CARD_IS_DPD_SUM,CARD_M3CARD_IS_DPD_UNDER_120_MEAN,CARD_M3CARD_IS_DPD_UNDER_120_SUM,CARD_M3CARD_IS_DPD_OVER_120_MEAN,CARD_M3CARD_IS_DPD_OVER_120_SUM
0,100002,1.0,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003,0.0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004,0.0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,0.0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100007,0.0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
### one-hotencoding
encoding_columns = []

#### one-hotencoding処理対象列を取得
for col in df.columns:
    if df[col].dtype=="O":
        print(col)
        encoding_columns.append(col)

NAME_CONTRACT_TYPE
CODE_GENDER
FLAG_OWN_CAR
FLAG_OWN_REALTY
NAME_TYPE_SUITE
NAME_INCOME_TYPE
NAME_EDUCATION_TYPE
NAME_FAMILY_STATUS
NAME_HOUSING_TYPE
OCCUPATION_TYPE
WEEKDAY_APPR_PROCESS_START
ORGANIZATION_TYPE
FONDKAPREMONT_MODE
HOUSETYPE_MODE
WALLSMATERIAL_MODE
EMERGENCYSTATE_MODE


In [10]:
# general cleaning procedures
df = df[df['CODE_GENDER'] != 'XNA']
df = df[df['AMT_INCOME_TOTAL'] < 20000000] # remove a outlier 117M
# NaN values for DAYS_EMPLOYED: 365.243 -> nan
df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True) # set null value
df['DAYS_LAST_PHONE_CHANGE'].replace(0, np.nan, inplace=True) # set null value

### CODE_GENDER, 
# Categorical features with Binary encode (0 or 1; two categories)
for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
    df[bin_feature], uniques = pd.factorize(df[bin_feature])

### NAME_INCOME_TYPE
df['NAME_INCOME_TYPE'].replace(['Maternity leave'], ['Unemployed'], inplace=True)
df['NAME_INCOME_TYPE'].replace(['Working','Commercial associate','Pensioner','State servant','Unemployed','Student','Businessman','Maternity leave'], [0, 1, 2, 3, 4, 5, 6, 7], inplace=True)
### NAME_FAMILY_STATUS
df['NAME_FAMILY_STATUS'].replace('Unknown', 'Married', inplace=True)
df['NAME_FAMILY_STATUS'].replace(['Married','Single / not married','Civil marriage','Separated','Widow'], [0, 1, 2, 3, 4], inplace=True)

### 
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '_', x))

df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CARD_M3BALANCE_LIMIT_RATIO_MIN,CARD_M3BALANCE_LIMIT_RATIO_MAX,CARD_M3DRAWING_LIMIT_RATIO_MIN,CARD_M3DRAWING_LIMIT_RATIO_MAX,CARD_M3CARD_IS_DPD_MEAN,CARD_M3CARD_IS_DPD_SUM,CARD_M3CARD_IS_DPD_UNDER_120_MEAN,CARD_M3CARD_IS_DPD_UNDER_120_SUM,CARD_M3CARD_IS_DPD_OVER_120_MEAN,CARD_M3CARD_IS_DPD_OVER_120_SUM
0,100002,1.0,Cash loans,0,0,0,0,202500.0,406597.5,24700.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003,0.0,Cash loans,1,0,1,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004,0.0,Revolving loans,0,1,0,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,0.0,Cash loans,1,0,0,0,135000.0,312682.5,29686.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100007,0.0,Cash loans,0,0,0,0,121500.0,513000.0,21865.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
#### one-hotencoding実行
for col in encoding_columns:
    df = pd.get_dummies(df, columns=[col], dummy_na=True)

df.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,WALLSMATERIAL_MODE_nan,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,EMERGENCYSTATE_MODE_nan
0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637.0,...,0,0,0,0,1,0,0,1,0,0
1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188.0,...,0,0,0,0,0,0,0,1,0,0
2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225.0,...,0,0,0,0,0,0,1,0,0,1
3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039.0,...,0,0,0,0,0,0,1,0,0,1
4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038.0,...,0,0,0,0,0,0,1,0,0,1


In [12]:
### infをnanへ置換
df = df.replace([np.inf, -np.inf], np.nan)

In [13]:
### 変数選択001
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 603 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 593 features are remained after removing non-informative features
60 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 533 features are remained after removing features not interesting for LightGBM classifier
---=> 38 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 571 features

---=> df final shape: (356250, 571)  <=--- 



,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,POS_M20MONTHS_BALANCE_MAX_high_risk,POS_M20MONTHS_BALANCE_MAX_low_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,CARD_M3CARD_IS_DPD_OVER_120_SUM_high_risk,CARD_M3CARD_IS_DPD_OVER_120_SUM_low_risk
0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637.0,...,0,1,0,0,0,0,0,0,0,0
1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188.0,...,1,0,0,0,0,0,0,0,0,0
2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225.0,...,0,0,0,0,0,0,0,0,0,0
3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3040.0,...,0,1,1,0,1,0,1,0,1,0
4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038.0,...,0,1,0,0,0,0,0,0,0,0


In [14]:
### 変数選択002
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 571 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 567 features are remained after removing non-informative features
22 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 545 features are remained after removing features not interesting for LightGBM classifier
---=> 15 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 560 features

---=> df final shape: (356250, 560)  <=--- 



,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,PREV_PREV_GOODS_DIFF_MAX_low_risk,BUREAU_BUREAU_IS_DPD_SUM_high_risk,BUREAU_BUREAU_IS_DPD_SUM_low_risk,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk
0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637.0,...,0,0,1,1,0,0,0,0,0,0
1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188.0,...,0,0,1,1,0,0,0,0,0,0
2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225.0,...,0,0,1,0,0,0,0,0,0,0
3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3040.0,...,0,0,0,0,1,0,1,0,1,0
4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038.0,...,0,0,1,0,0,0,0,0,0,0


In [15]:
# infをnanへ置換
df = df.replace([np.inf, -np.inf], np.nan)
df.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,PREV_PREV_GOODS_DIFF_MAX_low_risk,BUREAU_BUREAU_IS_DPD_SUM_high_risk,BUREAU_BUREAU_IS_DPD_SUM_low_risk,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk
0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637.0,...,0,0,1,1,0,0,0,0,0,0
1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188.0,...,0,0,1,1,0,0,0,0,0,0
2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225.0,...,0,0,1,0,0,0,0,0,0,0
3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3040.0,...,0,0,0,0,1,0,1,0,1,0
4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038.0,...,0,0,1,0,0,0,0,0,0,0


In [16]:
### 変数名から不正な文字を除去
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
print(df.shape)
df.head()

(356250, 560)


,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,PREV_PREV_GOODS_DIFF_MAX_low_risk,BUREAU_BUREAU_IS_DPD_SUM_high_risk,BUREAU_BUREAU_IS_DPD_SUM_low_risk,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk
0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637.0,...,0,0,1,1,0,0,0,0,0,0
1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188.0,...,0,0,1,1,0,0,0,0,0,0
2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225.0,...,0,0,1,0,0,0,0,0,0,0
3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3040.0,...,0,0,0,0,1,0,1,0,1,0
4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038.0,...,0,0,1,0,0,0,0,0,0,0


In [17]:
df_train, df_test = FeatureEngineering3.get_apps_all_train_test(df)
df_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,PREV_PREV_GOODS_DIFF_MAX_low_risk,BUREAU_BUREAU_IS_DPD_SUM_high_risk,BUREAU_BUREAU_IS_DPD_SUM_low_risk,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk
0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637.0,...,0,0,1,1,0,0,0,0,0,0
1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188.0,...,0,0,1,1,0,0,0,0,0,0
2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225.0,...,0,0,1,0,0,0,0,0,0,0
3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3040.0,...,0,0,0,0,1,0,1,0,1,0
4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038.0,...,0,0,1,0,0,0,0,0,0,0


In [18]:
df_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,PREV_PREV_GOODS_DIFF_MAX_low_risk,BUREAU_BUREAU_IS_DPD_SUM_high_risk,BUREAU_BUREAU_IS_DPD_SUM_low_risk,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk
307511,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,...,0,0,1,1,0,0,0,0,0,0
307512,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,...,0,0,1,1,0,0,0,0,0,0
307513,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,...,0,0,1,0,1,0,1,0,1,0
307514,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,...,0,0,1,1,1,0,1,0,1,0
307515,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
pred_train_1b = pd.read_csv('./home-credit-default-risk/exports/pred_train_1b.csv')
pred_test_1b =  pd.read_csv('./home-credit-default-risk/exports/pred_test_1b.csv')
pred_test_1b.head()

,SK_ID_CURR,pred_1b
0,100001,0.042863
1,100005,0.140563
2,100013,0.025033
3,100028,0.032262
4,100038,0.205365


In [20]:
df_train = df_train.merge(pred_train_1b, how='left', on='SK_ID_CURR')
df_test = df_test.merge(pred_test_1b, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,BUREAU_BUREAU_IS_DPD_SUM_high_risk,BUREAU_BUREAU_IS_DPD_SUM_low_risk,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,pred_1b
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,...,0,1,1,0,0,0,0,0,0,0.042863
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,...,0,1,1,0,0,0,0,0,0,0.140563
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,...,0,1,0,1,0,1,0,1,0,0.025033
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,...,0,1,1,1,0,1,0,1,0,0.032262
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,...,0,0,0,0,0,0,0,0,0,0.205365


In [21]:
pred_train_1a = pd.read_csv('./home-credit-default-risk/exports/pred_train_1a.csv')
pred_test_1a =  pd.read_csv('./home-credit-default-risk/exports/pred_test_1a.csv')
pred_test_1b.head()

,SK_ID_CURR,pred_1b
0,100001,0.042863
1,100005,0.140563
2,100013,0.025033
3,100028,0.032262
4,100038,0.205365


In [22]:
df_train = df_train.merge(pred_train_1a, how='left', on='SK_ID_CURR')
df_test = df_test.merge(pred_test_1a, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,BUREAU_BUREAU_IS_DPD_SUM_low_risk,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,pred_1b,pred_1a
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,...,1,1,0,0,0,0,0,0,0.042863,0.038227
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,...,1,1,0,0,0,0,0,0,0.140563,0.112768
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,...,1,0,1,0,1,0,1,0,0.025033,0.033638
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,...,1,1,1,0,1,0,1,0,0.032262,0.031213
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,...,0,0,0,0,0,0,0,0,0.205365,0.181109


In [23]:
train_set2a = pd.read_csv('./home-credit-default-risk/exports/train_set2a.csv')
test_set2a =  pd.read_csv('./home-credit-default-risk/exports/test_set2a.csv')
test_set2a.head()

,SK_ID_CURR,set2a
0,100001,0.042769
1,100005,0.103694
2,100013,0.036006
3,100028,0.048812
4,100038,0.173045


In [24]:
#df_train = df_train.merge(train_set2a, how='left', on='SK_ID_CURR')
#df_test = df_test.merge(test_set2a, how='left', on='SK_ID_CURR')
#df_test.head()

In [25]:
### infをnanへ置換
df_train = df_train.replace([np.inf, -np.inf], np.nan)
df_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,BUREAU_BUREAU_IS_DPD_SUM_low_risk,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,pred_1b,pred_1a
0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637.0,...,1,1,0,0,0,0,0,0,0.284542,0.162368
1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188.0,...,1,1,0,0,0,0,0,0,0.007952,0.027249
2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225.0,...,1,0,0,0,0,0,0,0,0.013653,0.030869
3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3040.0,...,0,0,1,0,1,0,1,0,0.033349,0.029480
4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038.0,...,1,0,0,0,0,0,0,0,0.071830,0.047569


In [26]:
### infをnanへ置換
df_test = df_test.replace([np.inf, -np.inf], np.nan)
df_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,BUREAU_BUREAU_IS_DPD_SUM_low_risk,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,pred_1b,pred_1a
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,...,1,1,0,0,0,0,0,0,0.042863,0.038227
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,...,1,1,0,0,0,0,0,0,0.140563,0.112768
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,...,1,0,1,0,1,0,1,0,0.025033,0.033638
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,...,1,1,1,0,1,0,1,0,0.032262,0.031213
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,...,0,0,0,0,0,0,0,0,0.205365,0.181109


In [27]:
### 処理対象の列名を配列化
#encoding_columns = []
encoding_columns = list(df_test.select_dtypes(include='number').columns)
encoding_columns.remove('SK_ID_CURR')

In [28]:
### indexを設定
df_train = df_train.set_index('SK_ID_CURR')
df_test = df_test.set_index('SK_ID_CURR')
df_train.head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,BUREAU_BUREAU_IS_DPD_SUM_low_risk,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,pred_1b,pred_1a
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637.0,-3648.0,...,1,1,0,0,0,0,0,0,0.284542,0.162368
100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188.0,-1186.0,...,1,1,0,0,0,0,0,0,0.007952,0.027249
100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225.0,-4260.0,...,1,0,0,0,0,0,0,0,0.013653,0.030869
100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3040.0,-9832.0,...,0,0,1,0,1,0,1,0,0.033349,0.029480
100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038.0,-4312.0,...,1,0,0,0,0,0,0,0,0.071830,0.047569


In [29]:
### 標準化
for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    result = modules.process_standardization(df_train, df_test, num_cols)
        
df_train.head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,BUREAU_BUREAU_IS_DPD_SUM_low_risk,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,pred_1b,pred_1a
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1.0,-0.577531,0.313221,-0.478099,-0.166152,-0.507239,-0.149780,1.506891,0.747070,0.379883,...,0.426973,0.645083,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,2.325571,0.986709
100003,0.0,-0.577531,0.933556,1.725433,0.592656,1.599963,-1.252930,-0.166811,0.511719,1.079102,...,0.426973,0.645083,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,-0.810405,-0.548398
100004,0.0,-0.577531,-0.927449,-1.152888,-1.404647,-1.091896,-0.783203,-0.689500,0.923828,0.206055,...,0.426973,-1.550188,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,-0.745770,-0.507271
100006,0.0,-0.577531,-0.307114,-0.711432,0.177857,-0.653403,-0.928711,-0.680105,-0.280518,-1.375000,...,-2.342067,-1.550188,1.693287,-0.138614,1.613777,0.0,1.613777,0.0,-0.522451,-0.523054
100007,0.0,-0.577531,-0.431181,-0.213740,-0.361753,-0.068746,0.562988,-0.892525,-0.279785,0.191284,...,0.426973,-1.550188,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,-0.086162,-0.317542


In [30]:
### indexを削除
df_train = df_train.reset_index()
df_test = df_test.reset_index()
df_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,BUREAU_BUREAU_IS_DPD_SUM_low_risk,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,pred_1b,pred_1a
0,100002,1.0,-0.577531,0.313221,-0.478099,-0.166152,-0.507239,-0.149780,1.506891,0.747070,...,0.426973,0.645083,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,2.325571,0.986709
1,100003,0.0,-0.577531,0.933556,1.725433,0.592656,1.599963,-1.252930,-0.166811,0.511719,...,0.426973,0.645083,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,-0.810405,-0.548398
2,100004,0.0,-0.577531,-0.927449,-1.152888,-1.404647,-1.091896,-0.783203,-0.689500,0.923828,...,0.426973,-1.550188,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,-0.745770,-0.507271
3,100006,0.0,-0.577531,-0.307114,-0.711432,0.177857,-0.653403,-0.928711,-0.680105,-0.280518,...,-2.342067,-1.550188,1.693287,-0.138614,1.613777,0.0,1.613777,0.0,-0.522451,-0.523054
4,100007,0.0,-0.577531,-0.431181,-0.213740,-0.361753,-0.068746,0.562988,-0.892525,-0.279785,...,0.426973,-1.550188,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,-0.086162,-0.317542


In [31]:
### indexを削除
df_train = df_train.drop(columns=['index'])
df_test = df_test.drop(columns=['index'])
df_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,BUREAU_BUREAU_IS_DPD_SUM_low_risk,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,pred_1b,pred_1a
0,100001,-0.577531,-0.307114,-0.075104,-0.451791,-0.239271,-0.146484,-0.734184,0.024017,-0.051636,...,0.426973,0.645083,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,-0.414584,-0.423673
1,100005,-0.577531,-0.637959,-0.934826,-0.671920,-0.970093,1.079102,-0.464476,-0.891113,-1.172852,...,0.426973,0.645083,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,0.693131,0.423197
2,100013,-0.577531,0.313221,0.159594,2.943903,0.247943,-0.127563,-0.916815,-0.886230,0.797852,...,0.426973,-1.550188,1.693287,-0.138614,1.613777,0.0,1.613777,0.0,-0.616737,-0.475807
3,100028,2.192099,1.347112,2.424818,1.511669,2.805819,0.399658,0.472284,0.221558,0.847656,...,0.426973,0.645083,1.693287,-0.138614,1.613777,0.0,1.613777,0.0,-0.534781,-0.503358
4,100038,0.807284,0.106443,0.065768,0.342100,0.235763,-0.783203,0.686768,0.082153,0.279785,...,-2.342067,-1.550188,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,1.427858,1.199636


In [32]:
### infをnanへ置換
df_train = df_train.replace([np.inf, -np.inf], np.nan)
df_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,BUREAU_BUREAU_IS_DPD_SUM_low_risk,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,pred_1b,pred_1a
0,100002,1.0,-0.577531,0.313221,-0.478099,-0.166152,-0.507239,-0.149780,1.506891,0.747070,...,0.426973,0.645083,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,2.325571,0.986709
1,100003,0.0,-0.577531,0.933556,1.725433,0.592656,1.599963,-1.252930,-0.166811,0.511719,...,0.426973,0.645083,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,-0.810405,-0.548398
2,100004,0.0,-0.577531,-0.927449,-1.152888,-1.404647,-1.091896,-0.783203,-0.689500,0.923828,...,0.426973,-1.550188,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,-0.745770,-0.507271
3,100006,0.0,-0.577531,-0.307114,-0.711432,0.177857,-0.653403,-0.928711,-0.680105,-0.280518,...,-2.342067,-1.550188,1.693287,-0.138614,1.613777,0.0,1.613777,0.0,-0.522451,-0.523054
4,100007,0.0,-0.577531,-0.431181,-0.213740,-0.361753,-0.068746,0.562988,-0.892525,-0.279785,...,0.426973,-1.550188,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,-0.086162,-0.317542


In [33]:
### infをnanへ置換
df_test = df_test.replace([np.inf, -np.inf], np.nan)
df_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,BUREAU_BUREAU_IS_DPD_SUM_low_risk,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,pred_1b,pred_1a
0,100001,-0.577531,-0.307114,-0.075104,-0.451791,-0.239271,-0.146484,-0.734184,0.024017,-0.051636,...,0.426973,0.645083,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,-0.414584,-0.423673
1,100005,-0.577531,-0.637959,-0.934826,-0.671920,-0.970093,1.079102,-0.464476,-0.891113,-1.172852,...,0.426973,0.645083,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,0.693131,0.423197
2,100013,-0.577531,0.313221,0.159594,2.943903,0.247943,-0.127563,-0.916815,-0.886230,0.797852,...,0.426973,-1.550188,1.693287,-0.138614,1.613777,0.0,1.613777,0.0,-0.616737,-0.475807
3,100028,2.192099,1.347112,2.424818,1.511669,2.805819,0.399658,0.472284,0.221558,0.847656,...,0.426973,0.645083,1.693287,-0.138614,1.613777,0.0,1.613777,0.0,-0.534781,-0.503358
4,100038,0.807284,0.106443,0.065768,0.342100,0.235763,-0.783203,0.686768,0.082153,0.279785,...,-2.342067,-1.550188,-0.590567,-0.138614,-0.619664,0.0,-0.619664,0.0,1.427858,1.199636


In [34]:
### メモリサイズを縮小
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 692.10 MB
Memory usage of optimization is 690043592.00 MB
Decreased by -99703483.3%
Memory usage of dataframe is 109.61 MB
Memory usage of optimization is 109186688.00 MB
Decreased by -99610179.0%


,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,BUREAU_BUREAU_IS_DPD_SUM_low_risk,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,pred_1b,pred_1a
0,100002,1.0,-0.577637,0.313232,-0.478027,-0.166138,-0.507324,-0.149780,1.506836,0.747070,...,0.427002,0.645020,-0.590332,-0.138672,-0.619629,0.0,-0.619629,0.0,2.326172,0.986816
1,100003,0.0,-0.577637,0.933594,1.725586,0.592773,1.599609,-1.252930,-0.166870,0.511719,...,0.427002,0.645020,-0.590332,-0.138672,-0.619629,0.0,-0.619629,0.0,-0.810547,-0.548340
2,100004,0.0,-0.577637,-0.927246,-1.153320,-1.404297,-1.091797,-0.783203,-0.689453,0.923828,...,0.427002,-1.549805,-0.590332,-0.138672,-0.619629,0.0,-0.619629,0.0,-0.745605,-0.507324
3,100006,0.0,-0.577637,-0.307129,-0.711426,0.177856,-0.653320,-0.928711,-0.680176,-0.280518,...,-2.341797,-1.549805,1.693359,-0.138672,1.614258,0.0,1.614258,0.0,-0.522461,-0.522949
4,100007,0.0,-0.577637,-0.431152,-0.213745,-0.361816,-0.068726,0.562988,-0.892578,-0.279785,...,0.427002,-1.549805,-0.590332,-0.138672,-0.619629,0.0,-0.619629,0.0,-0.086182,-0.317627


### modelを生成

In [35]:
# trainデータセット作成
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=='O':
        x_train[col] = x_train[col].astype('category')

In [36]:
# testデータセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [37]:
### training keras
# 1層目のモデル
# pred_train_1a, pred_train_1bは、学習データのクロスバリデーションでの予測値
# pred_test_1a, pred_test_1bは、テストデータの予測値
model_1b = Model1Xgb()
pred_train_1b, pred_test_1b = predict_cv(model_1b, x_train, y_train, x_test)

[13:28:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_state", "num_round", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-auc:0.77437	eval-auc:0.77338
[1]	train-auc:0.77842	eval-auc:0.77380
[2]	train-auc:0.78935	eval-auc:0.78433
[3]	train-auc:0.78992	eval-auc:0.78401
[4]	train-auc:0.79155	eval-auc:0.78535
[5]	train-auc:0.79302	eval-auc:0.78704
[6]	train-auc:0.79336	eval-auc:0.78692
[7]	train-auc:0.79359	eval-auc:0.78736
[8]	train-auc:0.79382	eval-auc:0.78779
[9]	train-auc:0.79394	eval-auc:0.78785
[10]	train-auc:0.79382	eval-auc:0.78762
[11]	train-auc:0.79407	eval-auc:0.78792
[12]	train-auc:0.79411	eval-auc:0.78804
[13]	train-auc:0.79417	eval-auc:0.78740
[14

[192]	train-auc:0.81265	eval-auc:0.79181
[193]	train-auc:0.81272	eval-auc:0.79182
[194]	train-auc:0.81283	eval-auc:0.79182
[195]	train-auc:0.81303	eval-auc:0.79182
[196]	train-auc:0.81321	eval-auc:0.79190
[197]	train-auc:0.81336	eval-auc:0.79193
[198]	train-auc:0.81349	eval-auc:0.79192
[199]	train-auc:0.81363	eval-auc:0.79192
[200]	train-auc:0.81370	eval-auc:0.79193
[201]	train-auc:0.81382	eval-auc:0.79197
[202]	train-auc:0.81397	eval-auc:0.79197
[203]	train-auc:0.81403	eval-auc:0.79198
[204]	train-auc:0.81415	eval-auc:0.79204
[205]	train-auc:0.81431	eval-auc:0.79203
[206]	train-auc:0.81443	eval-auc:0.79204
[207]	train-auc:0.81454	eval-auc:0.79204
[208]	train-auc:0.81468	eval-auc:0.79205
[209]	train-auc:0.81484	eval-auc:0.79205
[210]	train-auc:0.81500	eval-auc:0.79206
[211]	train-auc:0.81516	eval-auc:0.79208
[212]	train-auc:0.81530	eval-auc:0.79211
[213]	train-auc:0.81540	eval-auc:0.79211
[214]	train-auc:0.81559	eval-auc:0.79212
[215]	train-auc:0.81570	eval-auc:0.79213
[216]	train-auc:

[392]	train-auc:0.84808	eval-auc:0.79222
[393]	train-auc:0.84833	eval-auc:0.79224
[394]	train-auc:0.84845	eval-auc:0.79224
[395]	train-auc:0.84859	eval-auc:0.79223
[396]	train-auc:0.84881	eval-auc:0.79222
[397]	train-auc:0.84901	eval-auc:0.79224
[398]	train-auc:0.84923	eval-auc:0.79225
[399]	train-auc:0.84941	eval-auc:0.79226
[400]	train-auc:0.84960	eval-auc:0.79224
[401]	train-auc:0.84979	eval-auc:0.79225
[402]	train-auc:0.84999	eval-auc:0.79226
[403]	train-auc:0.85016	eval-auc:0.79226
[404]	train-auc:0.85035	eval-auc:0.79225
[405]	train-auc:0.85060	eval-auc:0.79224
[406]	train-auc:0.85078	eval-auc:0.79224
[407]	train-auc:0.85094	eval-auc:0.79226
[408]	train-auc:0.85110	eval-auc:0.79226
[409]	train-auc:0.85131	eval-auc:0.79224
[410]	train-auc:0.85149	eval-auc:0.79224
[411]	train-auc:0.85161	eval-auc:0.79224
[412]	train-auc:0.85178	eval-auc:0.79225
[413]	train-auc:0.85194	eval-auc:0.79225
[414]	train-auc:0.85209	eval-auc:0.79223
[415]	train-auc:0.85226	eval-auc:0.79226
[416]	train-auc:

[592]	train-auc:0.87792	eval-auc:0.79212
[593]	train-auc:0.87799	eval-auc:0.79212
[594]	train-auc:0.87811	eval-auc:0.79212
[595]	train-auc:0.87823	eval-auc:0.79211
[596]	train-auc:0.87839	eval-auc:0.79209
[597]	train-auc:0.87860	eval-auc:0.79210
[598]	train-auc:0.87867	eval-auc:0.79209
[599]	train-auc:0.87877	eval-auc:0.79209
[600]	train-auc:0.87892	eval-auc:0.79208
[601]	train-auc:0.87903	eval-auc:0.79209
[602]	train-auc:0.87911	eval-auc:0.79209
[603]	train-auc:0.87926	eval-auc:0.79210
[604]	train-auc:0.87936	eval-auc:0.79211
[605]	train-auc:0.87945	eval-auc:0.79210
[606]	train-auc:0.87961	eval-auc:0.79210
[607]	train-auc:0.87977	eval-auc:0.79209
[608]	train-auc:0.87985	eval-auc:0.79210
[609]	train-auc:0.87998	eval-auc:0.79210
[610]	train-auc:0.88013	eval-auc:0.79210
[611]	train-auc:0.88022	eval-auc:0.79209
[612]	train-auc:0.88032	eval-auc:0.79209
[613]	train-auc:0.88046	eval-auc:0.79209
[614]	train-auc:0.88055	eval-auc:0.79209
[615]	train-auc:0.88064	eval-auc:0.79209
[616]	train-auc:

[792]	train-auc:0.90102	eval-auc:0.79136
[793]	train-auc:0.90120	eval-auc:0.79138
[794]	train-auc:0.90130	eval-auc:0.79138
[795]	train-auc:0.90137	eval-auc:0.79138
[796]	train-auc:0.90143	eval-auc:0.79138
[797]	train-auc:0.90154	eval-auc:0.79140
[798]	train-auc:0.90167	eval-auc:0.79139
[799]	train-auc:0.90182	eval-auc:0.79138
[800]	train-auc:0.90195	eval-auc:0.79138
[801]	train-auc:0.90204	eval-auc:0.79138
[802]	train-auc:0.90211	eval-auc:0.79137
[803]	train-auc:0.90222	eval-auc:0.79137
[804]	train-auc:0.90233	eval-auc:0.79138
[805]	train-auc:0.90248	eval-auc:0.79137
[806]	train-auc:0.90255	eval-auc:0.79137
[807]	train-auc:0.90266	eval-auc:0.79138
[808]	train-auc:0.90275	eval-auc:0.79137
[809]	train-auc:0.90283	eval-auc:0.79136
[810]	train-auc:0.90299	eval-auc:0.79136
[811]	train-auc:0.90310	eval-auc:0.79136
[812]	train-auc:0.90320	eval-auc:0.79135
[813]	train-auc:0.90328	eval-auc:0.79136
[814]	train-auc:0.90341	eval-auc:0.79133
[815]	train-auc:0.90355	eval-auc:0.79132
[816]	train-auc:

...

[162]	train-auc:0.80920	eval-auc:0.78950
[163]	train-auc:0.80932	eval-auc:0.78955
[164]	train-auc:0.80939	eval-auc:0.78955
[165]	train-auc:0.80952	eval-auc:0.78957
[166]	train-auc:0.80963	eval-auc:0.78955
[167]	train-auc:0.80979	eval-auc:0.78956
[168]	train-auc:0.80987	eval-auc:0.78957
[169]	train-auc:0.81003	eval-auc:0.78960
[170]	train-auc:0.81016	eval-auc:0.78962
[171]	train-auc:0.81030	eval-auc:0.78966
[172]	train-auc:0.81038	eval-auc:0.78967
[173]	train-auc:0.81064	eval-auc:0.78971
[174]	train-auc:0.81079	eval-auc:0.78967
[175]	train-auc:0.81095	eval-auc:0.78968
[176]	train-auc:0.81114	eval-auc:0.78969
[177]	train-auc:0.81127	eval-auc:0.78975
[178]	train-auc:0.81137	eval-auc:0.78974
[179]	train-auc:0.81157	eval-auc:0.78977
[180]	train-auc:0.81170	eval-auc:0.78980
[181]	train-auc:0.81178	eval-auc:0.78982
[182]	train-auc:0.81187	eval-auc:0.78984
[183]	train-auc:0.81206	eval-auc:0.78986
[184]	train-auc:0.81222	eval-auc:0.78987
[185]	train-auc:0.81235	eval-auc:0.78988
[186]	train-auc:

[362]	train-auc:0.84354	eval-auc:0.78998
[363]	train-auc:0.84378	eval-auc:0.78997
[364]	train-auc:0.84397	eval-auc:0.78997
[365]	train-auc:0.84416	eval-auc:0.78997
[366]	train-auc:0.84431	eval-auc:0.78997
[367]	train-auc:0.84453	eval-auc:0.78999
[368]	train-auc:0.84475	eval-auc:0.78997
[369]	train-auc:0.84491	eval-auc:0.78995
[370]	train-auc:0.84505	eval-auc:0.78996
[371]	train-auc:0.84520	eval-auc:0.78996
[372]	train-auc:0.84541	eval-auc:0.78995
[373]	train-auc:0.84555	eval-auc:0.78995
[374]	train-auc:0.84573	eval-auc:0.78995
[375]	train-auc:0.84594	eval-auc:0.78993
[376]	train-auc:0.84618	eval-auc:0.78991
[377]	train-auc:0.84644	eval-auc:0.78991
[378]	train-auc:0.84663	eval-auc:0.78992
[379]	train-auc:0.84683	eval-auc:0.78993
[380]	train-auc:0.84697	eval-auc:0.78994
[381]	train-auc:0.84722	eval-auc:0.78994
[382]	train-auc:0.84747	eval-auc:0.78991
[383]	train-auc:0.84767	eval-auc:0.78990
[384]	train-auc:0.84788	eval-auc:0.78988
[385]	train-auc:0.84812	eval-auc:0.78988
[386]	train-auc:

[562]	train-auc:0.87529	eval-auc:0.78944
[563]	train-auc:0.87537	eval-auc:0.78943
[564]	train-auc:0.87544	eval-auc:0.78943
[565]	train-auc:0.87557	eval-auc:0.78942
[566]	train-auc:0.87563	eval-auc:0.78940
[567]	train-auc:0.87578	eval-auc:0.78940
[568]	train-auc:0.87596	eval-auc:0.78940
[569]	train-auc:0.87606	eval-auc:0.78938
[570]	train-auc:0.87621	eval-auc:0.78937
[571]	train-auc:0.87632	eval-auc:0.78936
[572]	train-auc:0.87645	eval-auc:0.78936
[573]	train-auc:0.87661	eval-auc:0.78936
[574]	train-auc:0.87669	eval-auc:0.78936
[575]	train-auc:0.87685	eval-auc:0.78934
[576]	train-auc:0.87696	eval-auc:0.78934
[577]	train-auc:0.87710	eval-auc:0.78933
[578]	train-auc:0.87726	eval-auc:0.78932
[579]	train-auc:0.87743	eval-auc:0.78934
[580]	train-auc:0.87757	eval-auc:0.78934
[581]	train-auc:0.87774	eval-auc:0.78934
[582]	train-auc:0.87789	eval-auc:0.78933
[583]	train-auc:0.87806	eval-auc:0.78930
[584]	train-auc:0.87819	eval-auc:0.78929
[585]	train-auc:0.87830	eval-auc:0.78928
[586]	train-auc:

[761]	train-auc:0.89971	eval-auc:0.78884
[762]	train-auc:0.89978	eval-auc:0.78884
[763]	train-auc:0.89990	eval-auc:0.78884
[764]	train-auc:0.89999	eval-auc:0.78883
[765]	train-auc:0.90015	eval-auc:0.78882
[766]	train-auc:0.90024	eval-auc:0.78883
[767]	train-auc:0.90041	eval-auc:0.78883
[768]	train-auc:0.90048	eval-auc:0.78883
[769]	train-auc:0.90067	eval-auc:0.78881
[770]	train-auc:0.90076	eval-auc:0.78881
[771]	train-auc:0.90090	eval-auc:0.78882
[772]	train-auc:0.90103	eval-auc:0.78880
[773]	train-auc:0.90118	eval-auc:0.78878
[774]	train-auc:0.90133	eval-auc:0.78878
[775]	train-auc:0.90147	eval-auc:0.78876
[776]	train-auc:0.90158	eval-auc:0.78876
[777]	train-auc:0.90168	eval-auc:0.78875
[778]	train-auc:0.90181	eval-auc:0.78874
[779]	train-auc:0.90191	eval-auc:0.78875
[780]	train-auc:0.90197	eval-auc:0.78875
[781]	train-auc:0.90212	eval-auc:0.78874
[782]	train-auc:0.90225	eval-auc:0.78875
[783]	train-auc:0.90237	eval-auc:0.78875
[784]	train-auc:0.90247	eval-auc:0.78875
[785]	train-auc:

[961]	train-auc:0.92207	eval-auc:0.78809
[962]	train-auc:0.92216	eval-auc:0.78808
[963]	train-auc:0.92234	eval-auc:0.78807
[964]	train-auc:0.92245	eval-auc:0.78807
[965]	train-auc:0.92255	eval-auc:0.78808
[966]	train-auc:0.92261	eval-auc:0.78808
[967]	train-auc:0.92269	eval-auc:0.78807
[968]	train-auc:0.92278	eval-auc:0.78806
[969]	train-auc:0.92294	eval-auc:0.78806
[970]	train-auc:0.92300	eval-auc:0.78805
[971]	train-auc:0.92316	eval-auc:0.78805
[972]	train-auc:0.92329	eval-auc:0.78805
[973]	train-auc:0.92341	eval-auc:0.78805
[974]	train-auc:0.92350	eval-auc:0.78805
[975]	train-auc:0.92369	eval-auc:0.78804
[976]	train-auc:0.92379	eval-auc:0.78804
[977]	train-auc:0.92394	eval-auc:0.78804
[978]	train-auc:0.92406	eval-auc:0.78803
[979]	train-auc:0.92412	eval-auc:0.78802
[980]	train-auc:0.92421	eval-auc:0.78801
[981]	train-auc:0.92433	eval-auc:0.78801
[982]	train-auc:0.92449	eval-auc:0.78800
[983]	train-auc:0.92454	eval-auc:0.78799
[984]	train-auc:0.92463	eval-auc:0.78799
[985]	train-auc:

In [38]:
pred_test_1b

array([0.03430966, 0.12755553, 0.04487229, ..., 0.00953401, 0.02181215,
       0.204454  ], dtype=float32)

In [39]:
pred_train_1b

array([0.2594051 , 0.0230797 , 0.03328961, ..., 0.03448826, 0.07583097,
       0.08171385], dtype=float32)

In [40]:
pred_1b = []
for a in np.array(pred_train_1b):
    pred_1b.append(a)

#pred_1b = np.array(pred_1b)
len(pred_1b)

307506

In [41]:
print(type(pred_1b))

<class 'list'>


In [42]:
### 予測値を特徴量としてデータフレームを作成
sk_id_curr = df_train['SK_ID_CURR'].copy().to_list()
train_x_2 = pd.DataFrame({'SK_ID_CURR': sk_id_curr, 'set3x': pred_1b})
print(train_x_2.shape)
train_x_2.head()

(307506, 2)


,SK_ID_CURR,set3x
0,100002,0.259405
1,100003,0.023080
2,100004,0.033290
3,100006,0.022860
4,100007,0.033131


In [43]:
train_x_2.to_csv(
    path_or_buf="./home-credit-default-risk/exports/train_set3xgb.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

In [44]:
pred_1b = []
for a in np.array(pred_test_1b):
    pred_1b.append(a)

#pred_1b = np.array(pred_1b)
len(pred_1b)

48744

In [45]:
### 予測値を特徴量としてデータフレームを作成
sk_id_curr = df_test['SK_ID_CURR'].copy().to_list()
test_x_2 = pd.DataFrame({'SK_ID_CURR': sk_id_curr, 'set3x': pred_1b})
print(test_x_2.shape)
test_x_2.head()

(48744, 2)


,SK_ID_CURR,set3x
0,100001,0.034310
1,100005,0.127556
2,100013,0.044872
3,100028,0.030915
4,100038,0.201295


In [46]:
test_x_2.to_csv(
    path_or_buf="./home-credit-default-risk/exports/test_set3xgb.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

In [47]:
del df
del x_train
del y_train
del x_test
del df_train
del df_test
del model_1b
gc.collect()

0

## 特徴量生成2